In [1]:
from langchain_openai import ChatOpenAI

# Use Ollama's local API via OpenAI-compatible interface
llm = ChatOpenAI(
    base_url="http://localhost:11434/v1",  # Ollama's default
    api_key="ollama",  # dummy key required by SDK, can be anything
    model="llama3-groq-tool-use:latest",     # or llama2, mixtral, etc. (must match your Ollama pulled model)
    temperature=0.0
)


In [2]:
result = llm.invoke("What is an agent?")


In [3]:
type(result)


langchain_core.messages.ai.AIMessage

In [4]:
from rich.markdown import Markdown
Markdown(result.content) 

An agent can refer to different concepts depending on the context. Here are a few common meanings:                 

 1 In Business: An agent is a person or entity that acts on behalf of another, often in a business transaction. For
   example, a real estate agent represents buyers and sellers, while a literary agent represents authors.          
 2 In Computer Science: In programming, an agent can be a software component that performs specific tasks or       
   interacts with other components to achieve a goal. Examples include web agents, which interact with websites, or
   chatbots, which interact with users.                                                                            
 3 In Philosophy and Ethics: An agent is often defined as a being capable of making decisions and taking actions   
   based on those decisions. This concept is central in fields like ethics, moral philosophy, and decision theory. 
 4 In Everyday Life: In general terms, an agent can be anyone who acts on behalf of another person or entity to    
   achieve a specific goal.                                                                                        

Which context are you interested in?

In [5]:
from langchain.tools import tool


In [6]:
@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}' and content: {content}"
type(write_email) 

langchain_core.tools.structured.StructuredTool

In [7]:
write_email.args


{'to': {'title': 'To', 'type': 'string'},
 'subject': {'title': 'Subject', 'type': 'string'},
 'content': {'title': 'Content', 'type': 'string'}}

In [8]:
Markdown(write_email.description)


Write and send an email.

In [9]:
# Connect tools to a chat model
model_with_tools = llm.bind_tools([write_email], tool_choice="any", parallel_tool_calls=False)

# The model will now be able to call tools
output = model_with_tools.invoke("Draft a response to my boss (boss@company.ai) about tomorrow's meeting") 

In [10]:
type(output)


langchain_core.messages.ai.AIMessage

In [11]:
output

AIMessage(content='Could you provide me with the details of what you want to include in your email?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 178, 'total_tokens': 196, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama3-groq-tool-use:latest', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-444', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--32acf08c-5db6-4b1e-bd68-6a30507d2397-0', usage_metadata={'input_tokens': 178, 'output_tokens': 18, 'total_tokens': 196, 'input_token_details': {}, 'output_token_details': {}})

In [12]:
Markdown(output.content) 

Could you provide me with the details of what you want to include in your email?

In [21]:
output.tool_calls

[]

In [16]:
# Extract tool calls and execute them
args = output.tool_calls[0]['args']
args

IndexError: list index out of range

In [14]:
# Call the tool
result = write_email.invoke(args)
Markdown(result)

NameError: name 'args' is not defined

In [22]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

In [23]:
class StateSchema(TypedDict): 
    request: str
    email: str 
workflow=StateGraph(StateSchema)

In [34]:
def write_email_node(state: StateSchema) -> StateSchema:
    # Invoke the LLM with tools
    output = model_with_tools.invoke(state['request'])

    # Check if tool calls exist
    if not output.tool_calls or len(output.tool_calls) == 0:
        print("⚠️ No tool calls were returned by the model.")
        return {"email": "No email generated. LLM did not call any tool."}

    # Extract args from tool call
    args = output.tool_calls[0]['args']
    
    # Call actual tool
    email = write_email.invoke(args)

    return {"email": email}


In [35]:
workflow=StateGraph(StateSchema)
workflow.add_node("write_email_node", write_email_node)
workflow.add_edge(START, "write_email_node")
workflow.add_edge("write_email_node", END)

app=workflow.compile()

In [36]:
from IPython.display import Markdown,Image,display
display(Image(url="https://langchain-ai.github.io/langgraph/assets/workflow.png"))

In [28]:
#display(Image(app.get_graph().draw_mermaid_png()))

In [37]:
app.invoke({"request": "Draft a response to my boss (boss@company.ai) about tomorrow's meeting"})


⚠️ No tool calls were returned by the model.


{'request': "Draft a response to my boss (boss@company.ai) about tomorrow's meeting",
 'email': 'No email generated. LLM did not call any tool.'}

In [39]:
from typing import Literal
from langgraph.graph import MessagesState
#from email_assistant.utils import show_graph 



In [40]:
def call_llm(state: MessagesState) -> MessagesState:
    """Run LLM"""

    output = model_with_tools.invoke(state["messages"])
    return {"messages": [output]}

def run_tool(state: MessagesState):
    """Performs the tool call"""

    result = []
    for tool_call in state["messages"][-1].tool_calls:
        observation = write_email.invoke(tool_call["args"])
        result.append({"role": "tool", "content": observation, "tool_call_id": tool_call["id"]})
    return {"messages": result}

def should_continue(state: MessagesState) -> Literal["run_tool", "__end__"]:
    """Route to tool handler, or end if Done tool called"""
    
    # Get the last message
    messages = state["messages"]
    last_message = messages[-1]
    
    # If the last message is a tool call, check if it's a Done tool call
    if last_message.tool_calls:
        return "run_tool"
    # Otherwise, we stop (reply to the user)
    return END

workflow = StateGraph(MessagesState)
workflow.add_node("call_llm", call_llm)
workflow.add_node("run_tool", run_tool)
workflow.add_edge(START, "call_llm")
workflow.add_conditional_edges("call_llm", should_continue, {"run_tool": "run_tool", END: END})
workflow.add_edge("run_tool", END)

# Run the workflow
app = workflow.compile()

In [41]:
result = app.invoke({"messages": [{"role": "user", "content": "Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!"}]})
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!
================================== Ai Message ==================================

What would you like the subject of the email to be?


In [48]:
from langgraph.prebuilt import create_react_agent

In [49]:
agent=create_react_agent(
    model=llm,
    tools=[write_email],
    prompt="Respond to the user's request using the tools provided."
)

result=agent.invoke({"messages": [{"role": "user", "content": "Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!"}]})   

In [50]:
for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!
================================== Ai Message ==================================

What would you like the subject of the email to be?


# Memory to agent

In [53]:
from langgraph.checkpoint.memory import InMemorySaver


In [55]:
agent=create_react_agent(
    model=llm,
    tools=[write_email],
    prompt="Respond to the user's request using the tools provided.",
    checkpointer=InMemorySaver()
) 

In [56]:
config = {"configurable": {"thread_id": "1"}}
result = agent.invoke({"messages": [{"role": "user", "content": "What are some good practices for writing emails?"}]}, config)
                    

In [57]:
# Get the latest state snapshot
config = {"configurable": {"thread_id": "1"}}
state = agent.get_state(config)
for message in state.values['messages']:
    message.pretty_print()

================================ Human Message =================================

What are some good practices for writing emails?
================================== Ai Message ==================================

Here are some good practices for writing emails:

1. **Clear Subject Line**: Make sure the subject line is concise and accurately reflects the content of the email.
2. **Professional Tone**: Use a professional tone, avoiding slang or overly casual language.
3. **Clear Content**: Keep your message clear and to the point. Avoid using jargon or technical terms unless necessary.
4. **Polite Language**: Be polite and respectful in your language. Use phrases like "Thank you" or "Best regards."
5. **Proofread**: Always proofread your email for spelling, grammar, and punctuation errors before sending it.
6. **Use Proper Signatures**: Include a professional signature with your name, title, company, and contact information.
7. **Keep It Short**: Keep the email concise and to the point. 

In [59]:
# Continue the conversation
result = agent.invoke({"messages": [{"role": "user", "content": "Good, let's use lesson 3 to craft a response to my boss confirming that I want to attend Interrupt"}]}, config)
for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

What are some good practices for writing emails?
================================== Ai Message ==================================

Here are some good practices for writing emails:

1. **Clear Subject Line**: Make sure the subject line is concise and accurately reflects the content of the email.
2. **Professional Tone**: Use a professional tone, avoiding slang or overly casual language.
3. **Clear Content**: Keep your message clear and to the point. Avoid using jargon or technical terms unless necessary.
4. **Polite Language**: Be polite and respectful in your language. Use phrases like "Thank you" or "Best regards."
5. **Proofread**: Always proofread your email for spelling, grammar, and punctuation errors before sending it.
6. **Use Proper Signatures**: Include a professional signature with your name, title, company, and contact information.
7. **Keep It Short**: Keep the email concise and to the point. 

In [60]:
# Continue the conversation
result = agent.invoke({"messages": [{"role": "user", "content": "I like this, let's write the email to boss@company.ai"}]}, config)
for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

What are some good practices for writing emails?
================================== Ai Message ==================================

Here are some good practices for writing emails:

1. **Clear Subject Line**: Make sure the subject line is concise and accurately reflects the content of the email.
2. **Professional Tone**: Use a professional tone, avoiding slang or overly casual language.
3. **Clear Content**: Keep your message clear and to the point. Avoid using jargon or technical terms unless necessary.
4. **Polite Language**: Be polite and respectful in your language. Use phrases like "Thank you" or "Best regards."
5. **Proofread**: Always proofread your email for spelling, grammar, and punctuation errors before sending it.
6. **Use Proper Signatures**: Include a professional signature with your name, title, company, and contact information.
7. **Keep It Short**: Keep the email concise and to the point. 